In [ ]:
#default_exp core

In [ ]:
#export
from fastai_local.test import *
from fastai_local.imports import *
from itertools import zip_longest

# Core

> Basic functions used in the fastai library

## Type checking

In [ ]:
#export core
def chk(f): return typechecked(always=True)(f)

Runtime type checking is handy, so let's make it easy!

In [ ]:
show_doc(chk)

<h4 id="chk" class="doc_header"><code>chk</code><a class="source_link" data-toggle="collapse" data-target="#chk-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>chk</code>(**`f`**)

<div class="collapse" id="chk-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#chk-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>chk</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Decorator for a function to check that type-annotated arguments receive arguments of the right type.

In [ ]:
@chk
def test_chk(a:int=1): pass

In [ ]:
test_chk(1)
test_chk()

In [ ]:
test_fail(lambda: test_chk('a'), contains='"a" must be int')

## Monkey-patching

In [ ]:
#export
Tensor.ndim = property(lambda x: x.dim())

#### `Tensor.ndim`

We add an `ndim` property to `Tensor` with same semantics as [numpy ndim](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.ndim.html), which allows tensors to be used in matplotlib and other places that assume this property exists.

## Utility functions

In [ ]:
# export
def ifnone(a, b):
    "`b` if `a` is None else `a`"
    return b if a is None else a

Since `b if a is None else a` is such a common pattern, we wrap it in a function. However, be careful, because python will evaluate *both* `a` and `b` when calling `ifnone` (which it doesn't do if using the `if` version directly).

In [ ]:
test_eq(ifnone(None,1), 1)
test_eq(ifnone(2   ,1), 2)

In [ ]:
# export
def noop (x, *args, **kwargs):
    "Do nothing"
    return x

In [ ]:
test_eq(noop(1),1)

In [ ]:
# export
def noops(self, x, *args, **kwargs):
    "Do nothing (method)"
    return x

In [ ]:
class _t(): foo=noops
test_eq(_t().foo(1),1)

In [ ]:
#export
def range_of(x):
    "All indices of collection `x` (i.e. `list(range(len(x)))`)"
    return list(range(len(x)))

In [ ]:
test_eq(range_of([1,1,1,1]), [0,1,2,3])

In [ ]:
#export
def is_listy(x):
    "`isinstance(x, (tuple,list))`"
    return isinstance(x, (tuple,list))

In [ ]:
assert is_listy([1])
assert not is_listy(torch.tensor([1]))

In [ ]:
# export
def is_iter(o):
    "Test whether `o` can be used in a `for` loop"
    #Rank 0 tensors in PyTorch are not really iterable
    return isinstance(o, (Iterable,Generator)) and getattr(o,'ndim',1)

In [ ]:
assert is_iter([1])
assert not is_iter(torch.tensor(1))
assert is_iter(torch.tensor([1,2]))
assert (o for o in range(3))

In [ ]:
#export
def listify(o):
    "Make `o` a list."
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, (str,np.ndarray,Tensor)): return [o]
    if is_iter(o): return list(o)
    return [o]

In [ ]:
# test
test_eq(listify(None),[])
test_eq(listify([1,2,3]),[1,2,3])
test_ne(listify([1,2,3]),[1,2,])
test_eq(listify('abc'),['abc'])
test_eq(listify(range(0,3)),[0,1,2])
test_eq(listify(o for o in range(0,3)),[0,1,2])
test_eq(listify(torch.tensor(0)),[torch.tensor(0)])
test_eq(listify([torch.tensor(0),torch.tensor(1)]),[torch.tensor(0),torch.tensor(1)])
test_eq(listify(torch.tensor([0.,1.1]))[0],torch.tensor([0.,1.1]))

In [ ]:
#export
def tuplify(o):
    "Make `o` a tuple"
    return tuple(listify(o))

In [ ]:
test_eq(tuplify(None),())
test_eq(tuplify([1,2,3]),(1,2,3))

In [ ]:
#export
def tensor(x, *rest):
    "Like `torch.as_tensor`, but handle lists too, and can pass multiple vector elements directly."
    if len(rest): x = tuplify(x)+rest
    # Pytorch bug in dataloader using num_workers>0
    if is_listy(x) and len(x)==0: return tensor(0)
    res = torch.tensor(x) if is_listy(x) else as_tensor(x)
    if res.dtype is torch.int32:
        warn('Tensor is int32: upgrading to int64; for better performance use int64 input')
        return res.long()
    return res

In [ ]:
test_eq(tensor(array([1,2,3])), torch.tensor([1,2,3]))
test_eq(tensor(1,2,3), torch.tensor([1,2,3]))

In [ ]:
# export
@chk
def compose(*funcs: Callable):
    "Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all"
    def _inner(x, *args, **kwargs):
        for f in listify(funcs): x = f(x, *args, **kwargs)
        return x
    return _inner

In [ ]:
f1 = lambda o,p=0: (o*2)+p
f2 = lambda o,p=1: (o+1)/p
test_eq(f2(f1(3)), compose(f1,f2)(3))
test_eq(f2(f1(3,p=3),p=3), compose(f1,f2)(3,p=3))
test_eq(f2(f1(3,  3),  3), compose(f1,f2)(3,  3))

In [ ]:
# export
def mask2idxs(mask):
    "Convert bool mask list to index list"
    mask = list(mask)
    if isinstance(mask[0],bool): return [i for i,m in enumerate(mask) if m]
    return [int(i) for i in mask]

In [ ]:
test_eq(mask2idxs([False,True,False,True]), [1,3])
test_eq(mask2idxs(tensor([1,2,3])), [1,2,3])

In [ ]:
#export
def uniqueify(x, sort=False, bidir=False, start=None):
    "Return the unique elements in `x`, optionally `sort`-ed, optionally return the reverse correspondance."
    res = list(OrderedDict.fromkeys(x).keys())
    if start is not None: res = listify(start)+res
    if sort: res.sort()
    if bidir: return res, {v:k for k,v in enumerate(res)}
    return res

In [ ]:
# test
test_eq(set(uniqueify([1,1,0,5,0,3])),{0,1,3,5})
test_eq(uniqueify([1,1,0,5,0,3], sort=True),[0,1,3,5])
v,o = uniqueify([1,1,0,5,0,3], bidir=True)
test_eq(v,[1,0,5,3])
test_eq(o,{1:0, 0: 1, 5: 2, 3: 3})
v,o = uniqueify([1,1,0,5,0,3], sort=True, bidir=True)
test_eq(v,[0,1,3,5])
test_eq(o,{0:0, 1: 1, 3: 2, 5: 3})

In [ ]:
# export
def setify(o): return o if isinstance(o,set) else set(listify(o))

In [ ]:
# test
test_eq(setify(None),set())
test_eq(setify('abc'),{'abc'})
test_eq(setify([1,2,2]),{1,2})
test_eq(setify(range(0,3)),{0,1,2})
test_eq(setify({1,2}),{1,2})

In [ ]:
#export
def all_equal(a,b): return all(equals(a_,b_) for a_,b_ in zip_longest(a,b))

In [ ]:
test_eq([3,4],torch.tensor([3,4]))
test_ne([3,4],torch.tensor([3]))
test_ne([3,4],torch.tensor([3,5]))

In [ ]:
#export
def make_cross_image(bw=True):
    "Create a tensor containing a cross image, either `bw` (True) or color"
    if bw:
        im = torch.zeros(5,5)
        im[2] = 1.
        im[:,2] = 1.
    else:
        im = torch.zeros(3,5,5)
        im[0,2] = 1.
        im[1,:,2] = 1.
    return im

## Export-

In [ ]:
#hide
from fastai_local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 02_data_pipeline.ipynb.
Converted 03_data_source.ipynb.
Converted 05_data_core.ipynb.
Converted 06_pets_tutorial.ipynb.
Converted 99_export.ipynb.
Converted 99a_export2html.ipynb.
Converted _07_data_blocks.ipynb.
